In [ ]:
import pandas as pd
import numpy as np
import json
import requests
from pathlib import Path

pd.set_option('display.max_columns', 300)

In [ ]:
teams = ['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Ipswich Town', 'Leicester City', 'Liverpool', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Nottingham Forest', 'Southampton', 'Tottenham', 'West Ham', 'Wolves']

In [ ]:
# fetch FPL data online
fpl_online_data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
fpl_online_df = pd.DataFrame(fpl_online_data['elements'])
fpl_online_df['team_name'] = [teams[i] for i in fpl_online_df['team']-1]
fpl_online_df['name'] = fpl_online_df.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_online_df['season'] = '24-25'
fpl_online_df

In [ ]:
# save online data
my_path = Path('../data/fpl_df_preseason_online.csv')
fpl_online_df.to_csv(my_path)

In [ ]:
# fetch old data
my_path = Path('../data/modeling/fpl_df.csv')
fpl_df = pd.read_csv(my_path, index_col=0)
fpl_df 

In [ ]:
fpl_df_latest = fpl_df.groupby(['first_name', 'second_name']).last().reset_index()
fpl_df_latest['name'] = fpl_df_latest.apply(lambda x: x['first_name'] + ' ' + x['second_name'], axis=1)
fpl_df_latest

In [ ]:
fpl_df_latest[fpl_df_latest.name.str.contains('Jesus')]

In [ ]:
fpl_df_latest = fpl_df_latest.drop(361) # drop G.Jesus extra row from man city time

In [ ]:
# only keep players that are found this season
fpl_df_latest = fpl_df_latest[fpl_df_latest['name'].isin(fpl_online_df.name.unique())].reset_index()
fpl_df_latest

In [ ]:
correct_teams = []
correct_positions = []
for ix, row in fpl_df_latest.iterrows():
    my_name = row['name']
    my_position = row.element_type
    online_team = fpl_online_df.loc[fpl_online_df.name==my_name, 'team'].item()
    online_position = fpl_online_df.loc[fpl_online_df.name==my_name, 'element_type'].item()
    correct_teams.append(online_team)
    correct_positions.append(online_position)

fpl_df_latest['team'] = correct_teams
fpl_df_latest['element_type'] = correct_positions

fpl_df_latest

In [ ]:
fpl_df_latest['team_name'] = [teams[i] for i in fpl_df_latest['team']-1]
fpl_df_latest

In [ ]:
fpl_df_latest[fpl_df_latest.name.str.contains('Wood')]

In [ ]:
# save preseason player data
my_path = Path('../data/fpl_df_preseason.csv')
fpl_df_latest.to_csv(my_path)

In [ ]:
# get FPL fixtures data
fpl_fixtures_data = json.loads(requests.get('https://fantasy.premierleague.com/api/fixtures/').text)
fpl_fixtures = pd.DataFrame(fpl_fixtures_data)
fpl_fixtures['home_team'] = [teams[i] for i in fpl_fixtures['team_h']-1]
fpl_fixtures['away_team'] = [teams[i] for i in fpl_fixtures['team_a']-1]
fpl_fixtures

In [ ]:
# save fpl fixtures
my_path = Path('../data/fixtures/fpl_fixtures.csv')
fpl_fixtures.to_csv(my_path)

In [ ]:
# team data, add estimate for new teams
my_path = Path('../data/team_data.csv')
team_data = pd.read_csv(my_path, index_col=0)
team_data 

In [ ]:
team_data['xG_ewm_5'].plot(kind='hist')

In [ ]:
team_data['xG_ewm_40'].plot(kind='hist')

In [ ]:
team_data['xGA_ewm_5'].plot(kind='hist')

In [ ]:
team_data['xGA_ewm_40'].plot(kind='hist')

In [ ]:
team_data.tail(20)

In [ ]:
[team for team in teams if team not in team_data.value.unique()]

In [ ]:
# add extra row and edit relevant columns to add necessary team data
#team_data = pd.concat((team_data, pd.DataFrame(team_data.iloc[-1]).T)).reset_index(drop=True)

In [ ]:
new_row = pd.DataFrame(
    data=np.array(['Ipswich Town', 1, 1, 1, 1, 1.8, 1.8, 1.8, 1.8,]).reshape(1,9), 
    index=[0],
    columns=['value','xG_ewm_5', 'xG_ewm_10', 'xG_ewm_20', 'xG_ewm_40', 'xGA_ewm_5', 'xGA_ewm_10', 'xGA_ewm_20', 'xGA_ewm_40']
)
new_row

In [ ]:
team_data = pd.concat((team_data, new_row), ignore_index=True)
team_data

In [ ]:
# save pre-season team data
my_path = Path('../data/team_data_preseason.csv')
team_data.to_csv(my_path)